#### Instructions:  
1. Libraries allowed: **Python basic libraries, numpy, pandas, scikit-learn (only for data processing), pytorch, and ClearML.**
2. Show all outputs.
3. Submit jupyter notebook and a pdf export of the notebook. Check canvas for detail instructions for the report.
4. Below are the questions/steps that you need to answer. Add as many cells as needed.


In [3]:
!pip install --quiet git+https://github.com/allegroai/clearml


In [4]:
%env CLEARML_WEB_HOST=https://app.clear.ml/
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY = W6U5DT7EO5OI92P270XMOHAVNUBLQW
%env CLEARML_API_SECRET_KEY = 8i9pBQJ3lQRslWEC4LtgCB7cFoW-V7BGTcWdAIWKVTJPqwaYALQ7dvQMccFQR1DDVFE

env: CLEARML_WEB_HOST=https://app.clear.ml/
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=W6U5DT7EO5OI92P270XMOHAVNUBLQW
env: CLEARML_API_SECRET_KEY=8i9pBQJ3lQRslWEC4LtgCB7cFoW-V7BGTcWdAIWKVTJPqwaYALQ7dvQMccFQR1DDVFE


In [5]:
import clearml

clearml.browser_login()

In [6]:
import logging
from clearml import Task
from clearml.automation import (
    DiscreteParameterRange, HyperParameterOptimizer, RandomSearch,
    UniformParameterRange, LogUniformParameterRange, UniformIntegerParameterRange)


In [7]:

try:
    from clearml.automation.optuna import OptimizerOptuna
    aSearchStrategy = OptimizerOptuna
except ImportError as ex:
    try:
        from clearml.automation.hpbandster import OptimizerBOHB
        aSearchStrategy = OptimizerBOHB
    except ImportError as ex:
        logging.getLogger().warning(
            'Apologies, it seems you do not have \'optuna\' or \'hpbandster\' installed, '
            'we will be using RandomSearch strategy instead')
        aSearchStrategy = RandomSearch


def job_complete_callback(
    job_id,
    objective_value,
    objective_iteration,
    job_parameters,
    top_performance_job_id
):
    print('Job completed!', job_id, objective_value, objective_iteration, job_parameters)
    if job_id == top_performance_job_id:
        print('WOOT WOOT we broke the record! Objective reached {}'.format(objective_value))



Apologies, it seems you do not have 'optuna' or 'hpbandster' installed, we will be using RandomSearch strategy instead


## Step 4: hyperparameter tuning without learning rate decay
Do hyperparater tuning with ClearML and copy the plots (e.g., parallel coordinates) from ClearML and visualize them here.

In [8]:
task = Task.init(project_name='PART 2 Project 1 without LRD',
                 task_name='Automatic Hyper-Parameter Optimization',
                 task_type=Task.TaskTypes.optimizer,
                 reuse_last_task_id=False)


args = {
    'template_task_id': '80938b97d6d44a0a901a8242030d5573',
    'run_as_service': False,
}
args = task.connect(args)


ClearML Task: created new task id=5654df37f8804d5ca92efeaabc316d02
ClearML results page: https://app.clear.ml/projects/6fc4af4d8e354802a615661df7b8c2e2/experiments/5654df37f8804d5ca92efeaabc316d02/output/log
2024-11-26 21:15:26,658 - clearml.Task - INFO - No repository found, storing script code instead
ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring


In [9]:
an_optimizer = HyperParameterOptimizer(
    base_task_id=args['template_task_id'],
    hyper_parameters=[
        UniformIntegerParameterRange('General/num_layers', min_value=3, max_value=12, step_size=1),
        UniformIntegerParameterRange('General/num_filters', min_value=16, max_value=128, step_size=16),
        LogUniformParameterRange('General/learning_rate', min_value=1.0e-5, max_value=1.0e-2),
       UniformParameterRange('General/momentum', min_value=0.7, max_value=0.99),
        DiscreteParameterRange('General/weight_decay', values=[0, 1.0e-4, 1.0e-3])
    ],
    objective_metric_title='Loss',
    objective_metric_series='Train',
    objective_metric_sign='min',
    max_number_of_concurrent_tasks=2,
    optimizer_class=aSearchStrategy,
    spawn_project=None,
    save_top_k_tasks_only=None,
    total_max_jobs=5,
    min_iteration_per_job=2,
    max_iteration_per_job=4,
)

In [10]:
if args['run_as_service']:
    task.execute_remotely(queue_name='services', exit_process=True)

an_optimizer.set_report_period(0.2)
an_optimizer.start_locally(job_complete_callback=job_complete_callback)
an_optimizer.set_time_limit(in_minutes= 60)
an_optimizer.wait()
top_exp = an_optimizer.get_top_experiments(top_k=2)
print([t.id for t in top_exp])
an_optimizer.stop()

print('Optimization Done')


Progress report #0 completed, sleeping for 0.25 minutes
2024-11-26 21:15:44,494 - clearml.automation.optimization - INFO - Creating new Task: {'General/num_layers': 12, 'General/num_filters': 96, 'General/learning_rate': 1.0132440917076855, 'General/momentum': 0.91224950058795, 'General/weight_decay': 0.0001}
2024-11-26 21:15:47,159 - clearml.automation.optimization - INFO - Creating new Task: {'General/num_layers': 9, 'General/num_filters': 96, 'General/learning_rate': 1.0214521992631094, 'General/momentum': 0.7892136825140557, 'General/weight_decay': 0.001}
Progress report #1 completed, sleeping for 0.2 minutes
Progress report #2 completed, sleeping for 0.2 minutes
Progress report #3 completed, sleeping for 0.2 minutes
Progress report #4 completed, sleeping for 0.2 minutes
Progress report #5 completed, sleeping for 0.2 minutes
Progress report #6 completed, sleeping for 0.2 minutes
Progress report #7 completed, sleeping for 0.2 minutes
Progress report #8 completed, sleeping for 0.2 mi

Progress report #137 completed, sleeping for 0.2 minutes
Progress report #138 completed, sleeping for 0.2 minutes
Progress report #139 completed, sleeping for 0.2 minutes
Progress report #140 completed, sleeping for 0.2 minutes
Progress report #141 completed, sleeping for 0.2 minutes
Progress report #142 completed, sleeping for 0.2 minutes
Progress report #143 completed, sleeping for 0.2 minutes
Progress report #144 completed, sleeping for 0.2 minutes
Progress report #145 completed, sleeping for 0.2 minutes
Progress report #146 completed, sleeping for 0.2 minutes
Progress report #147 completed, sleeping for 0.2 minutes
Progress report #148 completed, sleeping for 0.2 minutes
Progress report #149 completed, sleeping for 0.2 minutes
Progress report #150 completed, sleeping for 0.2 minutes
Progress report #151 completed, sleeping for 0.2 minutes
Progress report #152 completed, sleeping for 0.2 minutes
Progress report #153 completed, sleeping for 0.2 minutes
Progress report #154 completed,

Progress report #281 completed, sleeping for 0.2 minutes
Progress report #282 completed, sleeping for 0.2 minutes
Progress report #283 completed, sleeping for 0.06704156398773194 minutes
['2b2d2b277d114c009dcbbc2389f4f69f', 'e61c6cffad084df1a3f5e6c5feed43cb']
Optimization Done


In [11]:
task.close()

## Step 5: hyperparameter tuning with learning rate decay
Do hyperparater tuning with ClearML and copy the plots (e.g., parallel coordinates) from ClearML and visualize them here.

In [12]:
task = Task.init(project_name='Part 2 Project 1 with LR delay',
                 task_name='Automatic Hyper-Parameter Optimization',
                 task_type=Task.TaskTypes.optimizer,
                 reuse_last_task_id=False)


args = {
    'template_task_id': '80938b97d6d44a0a901a8242030d5573',
    'run_as_service': False,
}
args = task.connect(args)


ClearML Task: created new task id=cd2b0b99b1d84db8af028da4d5927d2f
ClearML results page: https://app.clear.ml/projects/eebccbdf8c5640c6a492056fe91e2ba3/experiments/cd2b0b99b1d84db8af028da4d5927d2f/output/log
ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring


In [14]:
an_optimizer = HyperParameterOptimizer(
    base_task_id=args['template_task_id'],
    hyper_parameters=[
        DiscreteParameterRange('General/num_layers',  values=[4]),
        DiscreteParameterRange('General/num_filters',  values=[32]),
        DiscreteParameterRange('General/learning_rate',  values=[1e-3]),
        DiscreteParameterRange('General/momentum',  values=[0.9]),
        DiscreteParameterRange('General/weight_decay', values=[1e-4]),
        UniformParameterRange('General/learning_rate_decay', min_value=0.8, max_value=1.0)
    ],
    objective_metric_title='Loss',
    objective_metric_series='Train',
    objective_metric_sign='min',
    max_number_of_concurrent_tasks=2,
    optimizer_class = aSearchStrategy,
    spawn_project=None,
    save_top_k_tasks_only=None,
    total_max_jobs=4,
    min_iteration_per_job=2,
    max_iteration_per_job=3,
)

In [15]:
if args['run_as_service']:
    task.execute_remotely(queue_name='services', exit_process=True)

an_optimizer.set_report_period(0.2)
an_optimizer.start_locally(job_complete_callback=job_complete_callback)
an_optimizer.set_time_limit(in_minutes= 20)
an_optimizer.wait()
top_exp = an_optimizer.get_top_experiments(top_k=2)
print([t.id for t in top_exp])
an_optimizer.stop()

print('Optimization Done')

Progress report #0 completed, sleeping for 0.25 minutes
Progress report #1 completed, sleeping for 0.2 minutes
Progress report #2 completed, sleeping for 0.2 minutes
Progress report #3 completed, sleeping for 0.2 minutes
Progress report #4 completed, sleeping for 0.2 minutes
Progress report #5 completed, sleeping for 0.2 minutes
Progress report #6 completed, sleeping for 0.2 minutes
Progress report #7 completed, sleeping for 0.2 minutes
Progress report #8 completed, sleeping for 0.2 minutes
Progress report #9 completed, sleeping for 0.2 minutes
Progress report #10 completed, sleeping for 0.2 minutes
Progress report #11 completed, sleeping for 0.2 minutes
Progress report #12 completed, sleeping for 0.2 minutes
Progress report #13 completed, sleeping for 0.2 minutes
Progress report #14 completed, sleeping for 0.2 minutes
Progress report #15 completed, sleeping for 0.2 minutes
Progress report #16 completed, sleeping for 0.2 minutes
Progress report #17 completed, sleeping for 0.2 minutes
P

In [16]:
task.close()

## Step 6: Evaluation
Evaluate the best model on test dataset and report accuracy, precision, recall, and F1 score.

In [18]:
import torch
import torch.nn as nn
import torch.utils.data as data
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [19]:
test_transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),         
])

# Load the test dataset
test_dir = "Dataset/Test"
test_dataset = datasets.ImageFolder(test_dir, transform=test_transform)

# DataLoader for the test dataset
test_loader = data.DataLoader(test_dataset, batch_size=16, shuffle=False)

In [20]:
class SimpleCNN(nn.Module):
    def __init__(self, num_layers=4, num_filters=32, kernel_size=3, input_channels=3, num_classes=2):
        super(SimpleCNN, self).__init__()
        self.layers = nn.ModuleList()
        in_channels = input_channels
        for _ in range(num_layers):
            self.layers.append(nn.Conv2d(in_channels, num_filters, kernel_size, padding=kernel_size // 2))
            self.layers.append(nn.ReLU())
            self.layers.append(nn.MaxPool2d(2))
            in_channels = num_filters
        self.fc = nn.Linear(num_filters * (224 // (2 ** num_layers)) ** 2, num_classes)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [24]:
# Load the best model
best_model_path = "best_model.pth"
model = SimpleCNN(num_layers=4, num_filters=32)
model.load_state_dict(torch.load(best_model_path))

<All keys matched successfully>

In [25]:
model.eval()

# Evaluation loop
all_labels = []
all_predictions = []

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predictions.cpu().numpy())

# Calculate performance metrics
accuracy = accuracy_score(all_labels, all_predictions)
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')
f1 = f1_score(all_labels, all_predictions, average='weighted')

# Print the results
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


Accuracy: 0.70
Precision: 0.81
Recall: 0.70
F1 Score: 0.67


## Step 7: Analysis
Provide a complete analysis of the whole process.


### Step 1: Hyperparameter Tuning Without Learning Rate Decay
In this step, ClearML was utilized to perform hyperparameter optimization on a basic CNN model. A range of hyperparameters, including the number of layers, filters, learning rate, momentum, and weight decay, were explored using Uniform and LogUniform parameter ranges. The goal was to minimize the training loss. Random Search was the fallback optimizer when advanced optimization strategies (Optuna or hpbandster) were unavailable.

#### Observations:
- The optimization successfully identified hyperparameter combinations that reduced the training loss.
- The parallel coordinates plot from ClearML provided insights into which parameters had the most significant impact on the loss metric.
- A total of 5 experiments were conducted, with two top-performing configurations saved.

### Step 2: Hyperparameter Tuning With Learning Rate Decay
This step introduced an additional hyperparameter for learning rate decay. This modification was aimed at observing the effect of gradually reducing the learning rate during training, which often improves convergence.

#### Observations:
- Learning rate decay helped refine the model's performance, especially in cases where overfitting might occur.
- The ClearML platform visualized the parameter correlations and the resulting performance metrics, helping identify the optimal setup.

### Step 3: Model Evaluation
The best model obtained from the hyperparameter tuning process was evaluated on the test dataset. Metrics such as accuracy, precision, recall, and F1 score were calculated to quantify the model's performance.

- Results:
  - Accuracy: 70%
  - Precision: 81%
  - Recall: 70%
  - F1 Score: 67%
  
The precision was higher than the recall, indicating that the model was more conservative in making predictions, possibly avoiding false positives at the cost of missing some true positives.


#### Insights
The experiment demonstrates the importance of systematic hyperparameter tuning and evaluation. By leveraging tools like ClearML, the process becomes transparent, reproducible, and efficient. Although the model's performance metrics are decent, further improvements could be made by addressing dataset limitations and exploring advanced optimization techniques.

#### Conclusion
The hyperparameter tuning process with and without learning rate decay provided valuable insights into the effects of different configurations. The model achieved reasonable performance, and the analysis of ClearML's visualizations guided the selection of the optimal setup.